# **Official documentation for the conversion algorithm of HyperReal Knowledge graph.**

HyperReal is the first knowledge graph completely dedicated to cultural symbols, the relationship between a symbol and its symbolic meaning and contexts of symbols.

## DBpedia conversion

###Files needed:

*   dbpedia1.tsv
*   dbpedia2.tsv

If the files are not available, you can run the following queries on DBpedia sparql portal to get the same results (a part from possible updates on DBpedia's end)

Query 1


```sql
    SELECT distinct ?rc ?context ?simul where {
?rc dbp:symbol ?simul;
dbp:type ?context .
FILTER NOT EXISTS { ?rc a yago:RailwayStation104049098 }
FILTER NOT EXISTS { ?rc a wikidata:Q719456 }
FILTER NOT EXISTS { ?rc dbp:type dbr:Public_company }
FILTER NOT EXISTS { ?rc dbp:symbolCaption ?something }
FILTER NOT EXISTS {?rc a <http://dbpedia.org/ontology/Company>}
FILTER ( ?context != <http://dbpedia.org/resource/Passivity_(engineering)> ) }
```

Query 2



```sql
SELECT distinct ?prop ?symbol ?redirected where {?prop a skos:Concept .
?prop rdfs:label ?label .
?symbol dct:subject ?prop  .
FILTER regex(?label, "symbols of", "i")
OPTIONAL {?symbol dbo:wikiPageRedirects ?redirected }
}
```




###Python libraries needed

*   rdflib
*   urllib
*   pandas
*   re

###Mapping rationale for the filtering (not needed for the code, just for clarification

*   mapping.xlsx

### Expected output

A turtle serialization, in the Simulation Ontology Schema, of symbolic knowledge extracted from DBpedia.

## Wordnet Conversion

###Files needed 

no file needed

###Python librares needed

*   rdflib
*   ntlk
*   re

###Expected output

A turtle serialization, in the Simulation Ontology Schema, of symbolic knowledge extracted from WordNet.

## Olderr's Dictionary

###Files needed

*  A converted txt version of Olderr's dictionary that replaces bold words with words that start and end with "@". For example **hello** becomes @hello@. This file unfortunately cannot be shared due to copyright reasons, but the code used to convert the txt file content into a turtle is shared.
*  contexttrue.csv

###Python libraries needed

*   rdflib
*   csv
*   re

###Expected output

A turtle serialization, in the Simulation Ontology Schema, of symbolic knowledge extracted from Olderr's dictionary of symbols.


In [ ]:
# needed installation

pip install rdflib

In [ ]:
#imports
import pandas as pd
import urllib.parse
import rdflib
from rdflib import URIRef, BNode, Literal, Graph
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
from rdflib import namespace
import re
import nltk
from nltk.corpus import wordnet as wn


#DPEDIA CONVERSION "dbpedia2.tsv" file

In [ ]:
df = pd.read_csv('dbpedia2.tsv', sep='\t', header=0)
df.head()

In [ ]:

word_regex_pattern = re.compile("[^A-Za-zí]+")

def camel(chars):
  chars = chars.replace("_", "")
  chars = chars.strip()
  words = word_regex_pattern.split(chars)
  return "".join(w.lower() if i is 0 else w.title() for i, w in enumerate(words))

#we parse every dbpedia url, extract the label by removing the first part of it
#we don't consider urls that contain "symbol" in the "symbol" column of the tsv
#because in some cases resources are the subject of themselves or there are
#other cases in which symbol_of_X is the subject of National_symbols_of_X and
#this information is negligible.
def processdata(a,b,c):
  alist = a.split("resource/")
  urlrc = alist[0] + "resource/" + urllib.parse.quote(alist[1])
  if "symbol" in b:
    return ("del", "del", "del", "del", "del")
  if "symbols_of_" in a:
    labelrc = a.split("symbols_of_")[1].replace("_", " ")
  elif "Symbols_of_" in a:
    labelrc = a.split("Symbols_of_")[1].replace("_", " ")
  if isinstance(c, float):
    blist = b.split("resource/")
    urlsim = blist[0]+"resource/"+urllib.parse.quote(blist[1])
    labelsim = b.split("resource/")[1].replace("_", " ")
  else:
    clist = c.split("resource/")
    urlsim = clist[0]+"resource/"+urllib.parse.quote(clist[1])
    labelsim = c.split("resource/")[1].replace("_", " ")
  return urlrc, labelrc, urlsim, labelsim



In [ ]:
result = [processdata(x, y, z) for x, y, z in zip(df['prop'], df['symbol'], df['redirected'])]

In [ ]:
result= list(set(result))

In [ ]:
result.remove(("del","del", "del", "del", "del"))

In [ ]:
g = Graph()
onturi = "https://w3id.org/simulation/ontology/"
kb = "https://w3id.org/simulation/data/"
wordneturi = "http://wordnet-rdf.princeton.edu/lemma/"
sim = Namespace("https://w3id.org/simulation/ontology/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
dbr = Namespace("http://dbpedia.org/resource/")
kburi = Namespace(kb)
g.bind("sim", sim)
g.bind("kb", kburi)
g.bind("owl", owl)
source = "http://dbpedia.org/"
contextl = "General or Unknown"
contexturi = kb+camel(contextl)
g.add((URIRef(contexturi), RDF.type, sim.Context))
g.add((URIRef(contexturi), RDFS.label, Literal("General or Unknown")))
g.add((URIRef(source), RDF.type, sim.Source))
for tup in result:
  if "emblem" not in tup[3].lower(): #again sometimes "emblem of X" is the subject
  #of National_symbols_of_x and this information is not useful for us
    rcuri = kb+camel(tup[1])
    g.add((URIRef(rcuri), OWL.sameAs , URIRef(tup[0])))
    suri = kb+camel(tup[3])
    g.add((URIRef(suri), OWL.sameAs , URIRef(tup[2])))
    simuri = kb+camel(tup[3])+"-"+camel(tup[1])
    g.add((URIRef(rcuri), RDF.type , sim.RealityCounterpart))
    g.add((URIRef(rcuri), RDFS.label , Literal(tup[1].strip())))
    g.add((URIRef(suri), RDF.type, sim.Simulacrum))
    g.add((URIRef(suri), RDFS.label , Literal(tup[3].strip())))
    g.add((URIRef(simuri), RDF.type, sim.Simulation))
    g.add((URIRef(simuri), RDFS.label , Literal("Simulation of "+ tup[1].strip() + " in " + contextl + " context as " + tup[3].strip())))
    g.add((URIRef(simuri), sim.hasRealityCounterpart, URIRef(rcuri)))
    g.add((URIRef(simuri), sim.hasSimulacrum, URIRef(suri)))
    g.add((URIRef(simuri), sim.hasContext, URIRef(contexturi)))
    g.add((URIRef(simuri), sim.hasSource, URIRef(source)))
    g.add((URIRef(source), sim.isSourceOf, URIRef(simuri)))
    g.add((URIRef(contexturi), sim.isContextOf, URIRef(simuri)))
    g.add((URIRef(rcuri), sim.isRealityCounterpartOf, URIRef(simuri)))
    g.add((URIRef(suri), sim.isSimulacrumOf, URIRef(simuri)))

In [ ]:
g.serialize(destination="yourfilename.ttl", format="turtle")

# DBPEDIA CONVERSION file "dbpedia1.tsv"

In [ ]:
heraldic = '''
Academic
Business
Christian and Scholarship
College, Medical
Cultural
Drill Sorority
Exclusive Engineering-based Fraternity
Honor
http://dbpedia.org/resource/Christian_Sorority
http://dbpedia.org/resource/Cultural_interest_fraternities_and_sororities
http://dbpedia.org/resource/Fraternities_and_sororities
http://dbpedia.org/resource/Fraternity
http://dbpedia.org/resource/Honor_society
http://dbpedia.org/resource/List_of_social_fraternities_and_sororities
http://dbpedia.org/resource/Professional_fraternities_and_sororities
http://dbpedia.org/resource/Service_fraternities_and_sororities
International, Professional
Inter-University and Collegiate Fraternity and Sorority
Leadership and Service
Leadership, Service
Professional
Professional and Social
Professional fraternity
Professional, Scholarship, Service
Professional, Social
Secret/Social
Service
Service and Cultural interest
Service and Social
Service, Social
Social
Social and Professional
Social and Service Multicultural Sorority
Social Fraternity
Social Society
Social/Spiritual
Social-professional fraternity
Social-Service
Socio Economic Service Fraternity
Socio Political Service Fraternity
Sororal and service organization
University'''
heraldiclist = heraldic.split("\n")[1:] #taken from the mapping file
heraldiclist = set(heraldiclist)

In [ ]:
deletelist = '''active
County
Drill/Service
Fraternal/military
Fthh
http://dbpedia.org/resource/Electromechanics
http://dbpedia.org/resource/Electronic_component
http://dbpedia.org/resource/Jasminum_grandiflorum
http://dbpedia.org/resource/Jasminum_sambac
http://dbpedia.org/resource/Light_rail
http://dbpedia.org/resource/Military_occupation
http://dbpedia.org/resource/MOSFET
http://dbpedia.org/resource/National_flag
http://dbpedia.org/resource/Odisha_Legislative_Assembly
http://dbpedia.org/resource/Online_newspaper
http://dbpedia.org/resource/Private_School_(film)
http://dbpedia.org/resource/Studentenverbindung
http://dbpedia.org/resource/Switch
http://dbpedia.org/resource/Transistor
International
Modern
National
other3
Personal
Personal Standard
Public
Regional
Representation
shrub
State
State of War Flag
Through station
War Flag'''.split("\n")
deletelist = set(deletelist) #taken from the mapping file

In [ ]:
df = pd.read_csv('dbpedia1.tsv', sep='\t', header=0)
df.head()

In [ ]:
def processdata2(a,b,c): #we parse the uris of wikidata if they are uris, if they are strings we analyze whether there are
#some separation characters like "," or if there are expressions like "and/or" to create multiple reality counterparts.
  if len(deletelist.intersection([b])) < 1:
    rclist = a.split("resource/")
    rc = rclist[0] + "resource/"+ urllib.parse.quote(rclist[1])
    labelrc = a.split("resource/")[1].replace("_", " ")
    if len(heraldiclist.intersection([b])) < 1:
      if "resource/" in b:
        if b == "http://dbpedia.org/resource/Faculty_(division)":
          context = None
          contextl = "Academic"
        else:
          contextlist = b.split("resource/")
          context = contextlist[0] + "resource/" + urllib.parse.quote(contextlist[1])
          contextl = b.split("resource/")[1].replace("_", " ")
      else:
        if b == "Law-based":
          context = None
          contextl = "Law"
        elif b == "Roman goddess for peace":
          context = None
          contextl = "Roman"
        else:
          context = None
          contextl = b.replace("_", " ")
    else:
      context = None
      contextl = "heraldic"
    if "resource/" in c:
      resourcelist = c.split("resource/")
      resource = resourcelist[0] + "resource/" + urllib.parse.quote(resourcelist[1])
      resourcel = c.split("resource/")[1].replace("_", " ")
      resourcels = [resourcel]
    else:
      c = c.replace(" and ", ", ").replace(" or ", ", ").replace(" y ", ", ")
      resourcels = c.split(",")
      resourcels = [x for x in resourcels if len(x) > 2]
      resource = None
    return rc, labelrc, context, contextl, resource, resourcels
  else:
    return "notvalid"

In [ ]:
result2 = [processdata2(x, y, z) for x, y, z in zip(df['rc'], df['context'], df['simul'])]

In [ ]:
result2 = [x for x in result2 if len(x) > 1]

In [ ]:
g = Graph()
onturi = "https://w3id.org/simulation/ontology/"
kb = "https://w3id.org/simulation/data/"
wordneturi = "http://wordnet-rdf.princeton.edu/lemma/"
sim = Namespace("https://w3id.org/simulation/ontology/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
dbr = Namespace("http://dbpedia.org/resource/")
kburi = Namespace(kb)
g.bind("sim", sim)
g.bind("kb", kburi)
g.bind("owl", owl)
source = "http://dbpedia.org/"
g.add((URIRef(source), RDF.type, sim.Source))
for tup in result2:
  if tup == "notvalid":
    continue
  rcuri = kb+camel(tup[1])
  g.add((URIRef(rcuri), RDF.type , sim.RealityCounterpart))
  g.add((URIRef(rcuri), RDFS.label , Literal(tup[1].strip())))
  g.add((URIRef(rcuri), OWL.sameAs , URIRef(tup[0])))
  contexturi = kb+camel(tup[3])
  contextl = tup[3].strip()
  g.add((URIRef(contexturi), RDF.type, sim.Context))
  g.add((URIRef(contexturi), RDFS.label , Literal(tup[3].strip())))
  if tup[2] is not None:
    g.add((URIRef(contexturi), OWL.sameAs, URIRef(tup[2])))
  for simcrum in tup[5]:
    suri = kb+camel(simcrum)
    if tup[4] is not None:
      g.add((URIRef(suri), OWL.sameAs , URIRef(tup[4])))
    simuri = kb+camel(simcrum)+"-"+camel(tup[1])
    g.add((URIRef(suri), RDF.type, sim.Simulacrum))
    g.add((URIRef(suri), RDFS.label , Literal(simcrum.strip())))
    g.add((URIRef(simuri), RDF.type, sim.Simulation))
    g.add((URIRef(simuri), RDFS.label , Literal("Simulation of "+ tup[1].strip() + " in " + contextl + " context as " + simcrum.strip())))
    g.add((URIRef(simuri), sim.hasRealityCounterpart, URIRef(rcuri)))
    g.add((URIRef(simuri), sim.hasSimulacrum, URIRef(suri)))
    g.add((URIRef(simuri), sim.hasContext, URIRef(contexturi)))
    g.add((URIRef(simuri), sim.hasSource, URIRef(source)))
    g.add((URIRef(source), sim.isSourceOf, URIRef(simuri)))
    g.add((URIRef(contexturi), sim.isContextOf, URIRef(simuri)))
    g.add((URIRef(rcuri), sim.isRealityCounterpartOf, URIRef(simuri)))
    g.add((URIRef(suri), sim.isSimulacrumOf, URIRef(simuri)))

In [ ]:
g.serialize(destination="yourfilename2.ttl", format="turtle")

# WORDNET conversion

In [ ]:
nltk.download("wordnet")

In [ ]:
symbols = dict()
for i in wn.all_synsets():
  definition = i.definition()
  if "(figurative)" in definition or "symbol of" in definition or "emblem of" in definition or "personification" in definition or "symbolizing" in definition:
    name = [str(lemma.name()) for lemma in i.lemmas()]
    symbols[name[0]] =i.definition()

In [ ]:
s = wn.synset("symbol.n.02")
e = wn.synset("emblem.n.02")

In [ ]:
for i in s.hyponyms():
  name = [str(lemma.name()) for lemma in i.lemmas()]
  symbols[name[0]] =i.definition()

for i in e.hyponyms():
  name = [str(lemma.name()) for lemma in i.lemmas()]
  symbols[name[0]] =i.definition()

In [ ]:
symbolic_phrases = ["symbol of", "symbolizing", "emblematic of", "emblem of", "symbol or ideal that serves as a", "personification of", "representing", "symbol for"]

In [ ]:
complexsymbol = {}
for el in symbols:
  complexsymbol[el] = dict()
  if ";" in symbols[el]:
    parts = symbols[el].split(";")
    idx = 1
    for part in parts:
      name = "part" + str(idx)
      complexsymbol[el][name] = dict()
      complexsymbol[el][name]["text"] = part
      idx +=1
  else:
    complexsymbol[el]["part1"] = dict()
    complexsymbol[el]["part1"]["text"] = symbols[el]

In [ ]:
for symb in complexsymbol:
  for part in complexsymbol[symb]:
    for symbphrase in symbolic_phrases:
      if symbphrase in complexsymbol[symb][part]["text"]:
        if "(" == complexsymbol[symb][part]["text"][0]:
          context = complexsymbol[symb][part]["text"].split(")", 1)[0][1:]
          complexsymbol[symb][part]["context"] = context
        realityc = complexsymbol[symb][part]["text"].split(symbphrase)[1]
        complexsymbol[symb][part][symbphrase] = realityc

In [ ]:
for symb in complexsymbol:
  for part in complexsymbol[symb]:
    for key in complexsymbol[symb][part].keys():
      if key in symbolic_phrases:
        if " and " in complexsymbol[symb][part][key]:
          complexsymbol[symb][part][key] = complexsymbol[symb][part][key].split(" and ")
        elif " or " in complexsymbol[symb][part][key]:
          complexsymbol[symb][part][key] = complexsymbol[symb][part][key].split(" or ")
        else:
          complexsymbol[symb][part][key] = [complexsymbol[symb][part][key]]

In [ ]:
onturi = "https://w3id.org/simulation/ontology/"
dizuri = "https://w3id.org/simulation/data/"
kb = "https://w3id.org/simulation/data/"
wordneturi = "http://wordnet-rdf.princeton.edu/lemma/"

In [ ]:
import re
word_regex_pattern = re.compile("[^A-Za-zí]+")

def camel(chars):
  chars = chars.replace("_", "")
  chars = chars.strip()
  words = word_regex_pattern.split(chars)
  return "".join(w.lower() if i is 0 else w.title() for i, w in enumerate(words))

In [ ]:
g = Graph()
sim = Namespace("https://w3id.org/simulation/ontology/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
wn = Namespace(wordneturi)
kburi = Namespace(kb)
g.bind("wn", wordneturi)
g.bind("sim", sim)
g.bind("kb", kburi)
g.bind("owl", owl)

emblematic = True
for symb in complexsymbol:
  emblematic = False
  for part in complexsymbol[symb]:
    if "emblem" in complexsymbol[symb][part]["text"]:
      emblematic = True
    for key in complexsymbol[symb][part].keys():
      if key in symbolic_phrases:
        g.add((URIRef(kb+camel(symb)), RDF.type, sim.Simulacrum))
        g.add((URIRef(kb+camel(symb)), RDFS.label, Literal(symb.replace("_", " ").strip())))
        g.add((URIRef(kb+camel(symb)), owl.sameAs, URIRef(wordneturi+symb.strip().replace("_", "%20"))))
        for realitycounterp in complexsymbol[symb][part][key]:
          if key == "personification of":
            oldrc = realitycounterp.strip()
            realitycounterp = realitycounterp.strip()+" (Personified)"
            g.add((URIRef(kb+camel(oldrc)), sim.hasPersonifiedVariant, URIRef(kb+camel(realitycounterp))))
            g.add((URIRef(kb+camel(realitycounterp)), sim.isPersonifiedVariantOf, (URIRef(kb+camel(oldrc)))))
          simulation = camel(symb).strip()+"-"+camel(realitycounterp).strip()
          if complexsymbol[symb][part].get("context"):
            context = complexsymbol[symb][part]["context"]
          else:
            context = "General or Unknown"
          g.add((URIRef(kb+camel(context)), RDF.type, sim.Context))
          g.add((URIRef(kb+camel(context)), RDFS.label, Literal(context)))

          if emblematic is True:
            g.add((URIRef(kb+simulation), RDF.type, sim.EmblematicSimulation))
            g.add((URIRef(kb+simulation), RDFS.label, Literal("Emblematic Simulation of "+ realitycounterp.strip() + " in " + context + " context as " + symb.replace("_", " ").strip())))
          else:
            g.add((URIRef(kb+simulation), RDF.type, sim.Simulation))
            g.add((URIRef(kb+simulation), RDFS.label, Literal("Simulation of "+ realitycounterp.strip() + " in " + context + " context as " + symb.replace("_", " ").strip())))
          g.add((URIRef(kb+camel(realitycounterp)), RDF.type, sim.RealityCounterpart))
          g.add((URIRef(kb+camel(realitycounterp)), RDFS.label, Literal(realitycounterp.strip())))
          g.add((URIRef(kb+simulation), sim.hasRealityCounterpart, URIRef(kb+camel(realitycounterp))))
          g.add((URIRef(kb+camel(realitycounterp)), sim.isRealityCounterpartOf, URIRef(kb+simulation)))
          g.add((URIRef(kb+simulation), sim.hasSimulacrum, URIRef(kb+camel(symb))))
          g.add((URIRef(kb+simulation), sim.hasSource, URIRef("http://wordnet-rdf.princeton.edu/")))
          g.add((URIRef("http://wordnet-rdf.princeton.edu/"), sim.isSourceOf, URIRef(kb+simulation)))
          g.add((URIRef(kb+camel(symb)), sim.isSimulacrumOf, URIRef(kb+simulation)))
          g.add((URIRef(kb+simulation), sim.hasContext, URIRef(kb+camel(context))))
          g.add((URIRef(kb+camel(context)), sim.isContextOf, URIRef(kb+simulation)))

In [ ]:
g.serialize(destination="yourfilenamehere.ttl", format="turtle")

# Olderr's Dictionary Conversion

In [ ]:
lines = open("dictionary.txt", "r").readlines()

In [ ]:
import csv
from csv import DictReader
realcontexts = dict()
with open("contexttrue.csv", "r", encoding="utf-8-sig") as tsvfile: #this file contains
#manually selected contexts from a list of contexts extracted from Olderr.
  contdict = csv.DictReader(tsvfile, delimiter=";")
  for row in contdict:
    if row["multiple"] == "yes":
      if "," in row["name"]:
       realcontexts[row["name"]] = row["name"].split(", ") 
      elif "/" in row["name"]:
        realcontexts[row["name"]] = row["name"].split("/")
    else:
      realcontexts[row["name"]] = []

In [ ]:
lemmas = {}
problems = []
contexts = set()
for term in lines:
  lemma = term.split("@")[1]
  lemmas[lemma.strip()] = {"contexts":[], "gcvariants":[]}
  try:
    text = term.split(lemma+"@", 1)[1]
    divisionincontexts = text.split("•")
    firstmeanings = divisionincontexts[0]
    if len(firstmeanings) > 1:
      lemmas[lemma.strip()]["contexts"].append({"general":firstmeanings.split(";")})
    divisionincontexts.remove(firstmeanings)
    for division in divisionincontexts:
      if "@" in division and "see " not in division:
        context = division.split("@")[1]
        meanings = division.split(context+"@", 1)[1]
        context = context.strip()
        contexts.add(context)
        if lemma.strip() not in context and context in realcontexts:
          if len(realcontexts[context]) > 1:
            for ct in realcontexts[context]:
              lemmas[lemma.strip()]["contexts"].append({ct:meanings.split(";")})
          else:
            lemmas[lemma.strip()]["contexts"].append({context:meanings.split(";")})
        else:
          lemmas[lemma.strip()]["gcvariants"].append({context:meanings.split(";")})

  except:
    problems.append({"line":lines.index(term), "content":term}) #any problems
    # at the end of this first conversion from txt to python data structure should
    # appear in the problems list.

In [ ]:
for lemma in lemmas:
  for context in lemmas[lemma]["contexts"]:
    for key in context:
      toberemovedelements = []
      for element in context[key]:
        if ":" in element:
          variant = element.split(":")[0]
          meanings = element.split(":")[1]
          toberemovedelements.append(element)
          context[key].append({variant.strip():meanings.split(",")})
      for el in toberemovedelements:
        context[key].remove(el)

In [ ]:
word_regex_pattern = re.compile("[^A-Za-z0-9]+")

def camel(chars):
  words = word_regex_pattern.split(chars)
  return "".join(w.lower() if i is 0 else w.title() for i, w in enumerate(words))

In [ ]:
def add_variant(g, simulacrum, varianttext, kburi, onturi):
  variant = varianttext.split("see @")[1].strip()
  varcorr = camel(variant)
  g.add((URIRef(kburi+simulacrum), URIRef(onturi+"hasVariant"), URIRef(kburi+varcorr)))
  g.add((URIRef(kburi+varcorr), URIRef(onturi+"isVariantOf"), URIRef(kburi+simulacrum)))

In [ ]:
onturi = "https://w3id.org/simulation/ontology/"
dizuri = "https://w3id.org/simulation/data/"
ont = Namespace("https://w3id.org/simulation/ontology/")
diz = Namespace(dizuri)
longrealitycounterparts = []
specificrel = []
thold = 35
g = Graph()
g.bind("sim", ont)
g.bind("kb", diz)
g.add((diz.DictionaryOfSymbols, RDF.type, ont.Source))
for simulacrum in lemmas:
  simulacrumcorr = simulacrum.strip().replace(" ", "_")
  if simulacrumcorr[0] == "-":
    simulacrumcorr = simulacrumcorr.replace("-", "")
  g.add((URIRef(dizuri+camel(simulacrumcorr)), RDF.type, ont.Simulacrum))
  g.add((URIRef(dizuri+camel(simulacrumcorr)), RDFS.label, Literal(simulacrum.strip())))
  for key in lemmas[simulacrum]["contexts"]:
    for context in key:
      if context == "general":
        contextcorr = "generalOrUnknown"
        for realitycounterp in key["general"]:
          if isinstance(realitycounterp, str):
            if ":" not in realitycounterp and "charm" not in realitycounterp and "," not in realitycounterp and "attribute of " not in realitycounterp and "associated with " not in realitycounterp and "related to " not in realitycounterp and "manifestation of" not in realitycounterp and "emblem of" not in realitycounterp and "a specific against " not in realitycounterp and "protection against " not in realitycounterp and "correspond to " not in realitycounterp and "corresponds to " not in realitycounterp and "cure for " not in realitycounterp and "antidote for" not in realitycounterp and "charm" not in realitycounterp:
              realitycounterpcorr = camel(realitycounterp.strip().replace("’", "'"))
              if "see @" not in realitycounterp:
                g.add((URIRef(dizuri+realitycounterpcorr), RDF.type, ont.RealityCounterpart))
                g.add((URIRef(dizuri+realitycounterpcorr), RDFS.label, Literal(realitycounterp.strip())))
                simulation = camel(simulacrumcorr)+"-"+realitycounterpcorr
                g.add((URIRef(dizuri+simulation), RDF.type, ont.Simulation))
                g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Simulation of " + realitycounterp.strip() + " in a General or Unknown Context" + " as " + simulacrum.strip())))
                g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+contextcorr)))
                g.add((URIRef(dizuri+contextcorr), RDF.type, ont.Context))
                g.add((URIRef(dizuri+contextcorr), RDFS.label, Literal(context.strip())))
                g.add((URIRef(dizuri+contextcorr), ont.isContextOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+camel(simulacrumcorr))))
                g.add((URIRef(dizuri+camel(simulacrumcorr)), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+realitycounterpcorr)))
                g.add((URIRef(dizuri+realitycounterpcorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
                g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
              else:
                add_variant(g, camel(simulacrumcorr), realitycounterp, dizuri, onturi)
            else:
              specificrel.append({"rc":realitycounterp.strip(), "cc":"General or Unknown", "context":contextcorr, "simulacrum":simulacrum.strip(), "simulacrumc":camel(simulacrumcorr)})
      else:
        contextcorr = camel(context.strip().replace("’","'"))
        for realitycounterp in key[context]:
          if isinstance(realitycounterp, str):
            if ":" not in realitycounterp and "charm" not in realitycounterp and "," not in realitycounterp and "attribute of " not in realitycounterp and "associated with " not in realitycounterp and "related to " not in realitycounterp and "manifestation of" not in realitycounterp and "emblem of" not in realitycounterp and "a specific against " not in realitycounterp and "protection against " not in realitycounterp and "correspond to " not in realitycounterp and "corresponds to " not in realitycounterp and "cure for " not in realitycounterp and "antidote for" not in realitycounterp and "charm" not in realitycounterp:
              realitycounterpcorr = camel(realitycounterp.strip().replace("’", "'"))
              if "see @" not in realitycounterp:
                g.add((URIRef(dizuri+realitycounterpcorr), RDF.type, ont.RealityCounterpart))
                g.add((URIRef(dizuri+realitycounterpcorr), RDFS.label, Literal(realitycounterp.strip())))
                simulation = camel(simulacrumcorr)+"-"+realitycounterpcorr
                g.add((URIRef(dizuri+simulation), RDF.type, ont.Simulation))
                g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Simulation of " + realitycounterp.strip() + " in a " + context.strip() + " context as " + simulacrum.strip())))
                g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+contextcorr)))
                g.add((URIRef(dizuri+contextcorr), RDF.type, ont.Context))
                g.add((URIRef(dizuri+contextcorr), RDFS.label, Literal(context.strip())))
                g.add((URIRef(dizuri+contextcorr), ont.isContextOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+camel(simulacrumcorr))))
                g.add((URIRef(dizuri+camel(simulacrumcorr)), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+realitycounterpcorr)))
                g.add((URIRef(dizuri+realitycounterpcorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
                g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
            else:
              specificrel.append({"rc":realitycounterp.strip(), "cc":context.strip(), "context":contextcorr, "simulacrum":simulacrum.strip(), "simulacrumc":camel(simulacrumcorr)})
          if isinstance(realitycounterp, dict):
            for generalvariant in realitycounterp:
              generalvariantcorr = camel(generalvariant.strip().replace("’", "'"))
              g.add((URIRef(dizuri+generalvariantcorr), RDF.type, ont.Simulacrum))
              g.add((URIRef(dizuri+generalvariantcorr), ont.isVariantOf, URIRef(dizuri+camel(simulacrumcorr))))
              g.add((URIRef(dizuri+camel(simulacrumcorr)), ont.hasVariant, URIRef(dizuri+generalvariantcorr)))
              g.add((URIRef(dizuri+generalvariantcorr), RDFS.label, Literal(generalvariant.strip())))
              for realitycounterp2 in realitycounterp[generalvariant]:
                if ":" not in realitycounterp2 and "charm" not in realitycounterp2 and "," not in realitycounterp2 and "attribute of " not in realitycounterp2 and "associated with " not in realitycounterp2 and "related to " not in realitycounterp2 and "manifestation of" not in realitycounterp2 and "emblem of" not in realitycounterp2 and "a specific against " not in realitycounterp2 and "protection against " not in realitycounterp2 and "correspond to " not in realitycounterp2 and "corresponds to " not in realitycounterp2 and "cure for " not in realitycounterp2 and "antidote for" not in realitycounterp2 and "charm" not in realitycounterp:
                  realitycounterpcorr = camel(realitycounterp2.strip().replace("’", "'"))
                  if "see @" not in realitycounterp2:
                    g.add((URIRef(dizuri+realitycounterpcorr), RDF.type, ont.RealityCounterpart))
                    g.add((URIRef(dizuri+realitycounterpcorr), RDFS.label, Literal(realitycounterp2.strip())))
                    simulation = generalvariantcorr+"-"+realitycounterpcorr
                    g.add((URIRef(dizuri+simulation), RDF.type, ont.Simulation))
                    g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Simulation of " + realitycounterp2.strip() + " in a " + context.strip() + " context " + " as " + generalvariant.strip()  )))
                    g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+contextcorr)))
                    g.add((URIRef(dizuri+contextcorr), RDF.type, ont.Context))
                    g.add((URIRef(dizuri+contextcorr), RDFS.label, Literal(context.strip())))
                    g.add((URIRef(dizuri+contextcorr), ont.isContextOf, URIRef(dizuri+simulation)))
                    g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+generalvariantcorr)))
                    g.add((URIRef(dizuri+generalvariantcorr), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
                    g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+realitycounterpcorr)))
                    g.add((URIRef(dizuri+realitycounterpcorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
                    g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
                    g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
                else:
                  specificrel.append({"rc":realitycounterp2.strip(), "cc":context.strip(), "context":contextcorr, "simulacrum":generalvariant.strip(), "simulacrumc":generalvariantcorr})
  if len(lemmas[simulacrum]["gcvariants"]) > 0:
    contextcorr = "generalOrUnknown"
    g.add((URIRef(dizuri+contextcorr), RDF.type, ont.Context))
    for key in lemmas[simulacrum]["gcvariants"]:
      for generalvariant in key:
        generalvariantcorr = camel(generalvariant.strip().replace("’", "'"))
        g.add((URIRef(dizuri+generalvariantcorr), RDF.type, ont.Simulacrum))
        g.add((URIRef(dizuri+generalvariantcorr), ont.isVariantOf, URIRef(dizuri+camel(simulacrumcorr))))
        g.add((URIRef(dizuri+camel(simulacrumcorr)), ont.hasVariant, URIRef(dizuri+generalvariantcorr)))
        g.add((URIRef(dizuri+generalvariantcorr), RDFS.label, Literal(generalvariant.strip())))
        for realitycounterp in key[generalvariant]:
          if ":" not in realitycounterp and "charm" not in realitycounterp and "," not in realitycounterp and "attribute of " not in realitycounterp and "associated with " not in realitycounterp and "related to " not in realitycounterp and "manifestation of" not in realitycounterp and "emblem of" not in realitycounterp and "a specific against " not in realitycounterp and "protection against " not in realitycounterp and "correspond to " not in realitycounterp and "corresponds to " not in realitycounterp and "cure for " not in realitycounterp and "antidote for" not in realitycounterp and "charm" not in realitycounterp:
            realitycounterpcorr = camel(realitycounterp.strip().replace("’", "'"))
            if "see @" not in realitycounterp:
              g.add((URIRef(dizuri+realitycounterpcorr), RDF.type, ont.RealityCounterpart))
              g.add((URIRef(dizuri+realitycounterpcorr), RDFS.label, Literal(realitycounterp.strip())))
              simulation = generalvariantcorr+"-"+realitycounterpcorr
              g.add((URIRef(dizuri+simulation), RDF.type, ont.Simulation))
              g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Simulation of " + realitycounterp.strip() + " in a General or Unknown Context as " + generalvariant.strip() )))
              g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+contextcorr)))
              g.add((URIRef(dizuri+contextcorr), RDF.type, ont.Context))
              g.add((URIRef(dizuri+contextcorr), RDFS.label, Literal("General or Unknown Context")))
              g.add((URIRef(dizuri+contextcorr), ont.isContextOf, URIRef(dizuri+simulation)))
              g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+generalvariantcorr)))
              g.add((URIRef(dizuri+generalvariantcorr), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
              g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+realitycounterpcorr)))
              g.add((URIRef(dizuri+realitycounterpcorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
              g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
              g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
              if len(realitycounterp) > thold:
                longrealitycounterparts.append(realitycounterp)
          else:
            specificrel.append({"rc":realitycounterp.strip(), "cc":"General or Unknown", "context":contextcorr, "simulacrum":generalvariant.strip(), "simulacrumc":generalvariantcorr})

In [ ]:
charmmapping = [("charm for protection from",{"simtype":"ProtectionSimulation","litsimtype":"Protection Simulation", "realityc":"charm", "additionalrel":"preventedRealityCounterpart", "ari":"isPreventedRealityCounterpartOf"}),
                ("charm for", {"simtype":"Simulation","litsimtype":"Simulation", "realityc":"charm", "additionalrel":"elicitedRealityCounterpart", "ari":"isElicitedRealityCounterpartOf"}),
                ("charm against", {"simtype":"ProtectionSimulation","litsimtype":"Protection Simulation", "realityc":"charm", "additionalrel":"preventedRealityCounterpart", "ari":"isPreventedRealityCounterpartOf"}),
                ("charm to restore", {"simtype":"Simulation","litsimtype":"Simulation", "realityc":"charm", "additionalrel":"restoredRealityCounterpart", "ari":"isRestoredRealityCounterpartOf"}),
                ("charm to ease", {"simtype":"Simulation","simtype":"Simulation", "litsimtype":"Simulation", "realityc":"charm", "additionalrel":"easedRealityCounterpart", "ari":"isEasedRealityCounterpartOf"}),
                ("charm to promote", {"simtype":"Simulation","simtype":"Simulation", "litsimtype":"Simulation", "realityc":"charm", "additionalrel":"elicitedRealityCounterpart", "ari":"isElicitedRealityCounterpartOf"})]
for r in specificrel:
  if "attribute of " in r["rc"]:
    attributelist = r["rc"].split("attribute of ", 1)[1].split(",")
    for attr in attributelist:
      rco = attr
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
        g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
        simulation = r["simulacrumc"]+"-"+rccorr
        g.add((URIRef(dizuri+simulation), RDF.type, ont.AttributeSimulation))
        g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Attribute Simulation of " + rco.strip() + " in a " + r["cc"] + " context as " + r["simulacrum"].strip())))
        g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
        g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
        g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
        g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+rccorr)))
        g.add((URIRef(dizuri+rccorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
        g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        if len(rco) > thold:
          longrealitycounterparts.append(realitycounterp)
  elif "associated with " in r["rc"]:
    associatedlist = r["rc"].split("associated with ", 1)[1].split(",")
    for assoc in associatedlist:
      rco = assoc
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
        g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
        simulation = r["simulacrumc"]+"-"+rccorr
        g.add((URIRef(dizuri+simulation), RDF.type, ont.AssociationSimulation))
        g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Association Simulation of " + rco.strip() + " in a " + r["cc"] + " context as " + r["simulacrum"].strip())))
        g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
        g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
        g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
        g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+rccorr)))
        g.add((URIRef(dizuri+rccorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
        g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        if len(rco) > thold:
          longrealitycounterparts.append(realitycounterp)
  elif "related to " in r["rc"]:
    relatedlist = r["rc"].split("related to ", 1)[1].split(",")
    for related in relatedlist:
      rco = related
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
        g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
        simulation = r["simulacrumc"]+"-"+rccorr
        g.add((URIRef(dizuri+simulation), RDF.type, ont.RelatednessSimulation))
        g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Relatedness Simulation of " + rco.strip() + " in a " + r["cc"] + " context as " + r["simulacrum"].strip())))
        g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
        g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
        g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
        g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+rccorr)))
        g.add((URIRef(dizuri+rccorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
        g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        if len(rco) > thold:
          longrealitycounterparts.append(realitycounterp)
  elif "manifestation of " in r["rc"]:
    manifestationlist =  r["rc"].split("manifestation of ", 1)[1].split(",")
    for mani in manifestationlist:
      rco = mani
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
        g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
        simulation = r["simulacrumc"]+"-"+rccorr
        g.add((URIRef(dizuri+simulation), RDF.type, ont.ManifestationSimulation))
        g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Manifestation Simulation of " + rco.strip() + " in a " + r["cc"] + " context as " + r["simulacrum"].strip())))
        g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
        g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
        g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
        g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+rccorr)))
        g.add((URIRef(dizuri+rccorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
        g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        if len(rco) > thold:
          longrealitycounterparts.append(realitycounterp)
  elif "emblem of " in r["rc"]:
    emblemlist = r["rc"].split("emblem of ", 1)[1].split(",")
    for emblem in emblemlist:
      rco = emblem
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
        g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
        simulation = r["simulacrumc"]+"-"+rccorr
        g.add((URIRef(dizuri+simulation), RDF.type, ont.EmblematicSimulation))
        g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Emblematic Simulation of " +  rco.strip()+ " in a " + r["cc"] + " context as " + r["simulacrum"].strip())))
        g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
        g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
        g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
        g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+rccorr)))
        g.add((URIRef(dizuri+rccorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
        g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        if len(rco) > thold:
          longrealitycounterparts.append(realitycounterp)
  elif "a specific against " in r["rc"]:
    protectionlist1 = r["rc"].split("a specific against ", 1)[1].split(",")
    for protect in protectionlist1:
      rco = protect
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
        g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
        simulation = r["simulacrumc"]+"-"+rccorr
        g.add((URIRef(dizuri+simulation), RDF.type, ont.ProtectionSimulation))
        g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Protection Simulation of " + r["simulacrum"].strip() + " in a " + r["cc"] + " context as a protection against " + rco.strip())))
        g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
        g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
        g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
        g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.preventedRealityCounterpart, URIRef(dizuri+rccorr)))
        g.add((URIRef(dizuri+rccorr), ont.isPreventedRealityCounterpartOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
        g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        if len(rco) > thold:
          longrealitycounterparts.append(realitycounterp)
  elif "protection against " in r["rc"]:
    protectionlist2 = r["rc"].split("protection against ", 1)[1].split(",")
    for protect in protectionlist2:
      rco = protect
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
        g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
        simulation = r["simulacrumc"]+"-"+rccorr
        g.add((URIRef(dizuri+simulation), RDF.type, ont.ProtectionSimulation))
        g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Protection Simulation of " + r["simulacrum"].strip() + " in a " + r["cc"] + " context as a protection against " + rco.strip())))
        g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
        g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
        g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
        g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.preventedRealityCounterpart, URIRef(dizuri+rccorr)))
        g.add((URIRef(dizuri+rccorr), ont.isPreventedRealityCounterpartOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
        g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        if len(rco) > thold:
          longrealitycounterparts.append(realitycounterp)
  elif "correspond to " in r["rc"]:
    correspondlist1 = r["rc"].split("correspond to ", 1)[1].split(",")
    for corr in correspondlist1:
      rco = corr
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
        g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
        simulation = r["simulacrumc"]+"-"+rccorr
        g.add((URIRef(dizuri+simulation), RDF.type, ont.CorrespondenceSimulation))
        g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Correspondence Simulation of " + rco.strip() + " in a " + r["cc"] + " context as " + r["simulacrum"].strip())))
        g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
        g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
        g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
        g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+rccorr)))
        g.add((URIRef(dizuri+rccorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
        g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        if len(rco) > thold:
          longrealitycounterparts.append(realitycounterp)
  elif "corresponds to " in r["rc"]:
    correspondlist2 = r["rc"].split("corresponds to ", 1)[1].split(",")
    for corr in correspondlist2:
      rco = corr
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
        g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
        simulation = r["simulacrumc"]+"-"+rccorr
        g.add((URIRef(dizuri+simulation), RDF.type, ont.CorrespondenceSimulation))
        g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Correspondence Simulation of " + rco.strip() + " in a " + r["cc"] + " context as " + r["simulacrum"].strip())))
        g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
        g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
        g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
        g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+rccorr)))
        g.add((URIRef(dizuri+rccorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
        g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        if len(rco) > thold:
          longrealitycounterparts.append(realitycounterp)
  elif "cure for " in r["rc"]:
    cureforlist = r["rc"].split("cure for ", 1)[1].split(",")
    for cure in cureforlist:
      rco = cure
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
        g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
        simulation = r["simulacrumc"]+"-"+rccorr
        g.add((URIRef(dizuri+simulation), RDF.type, ont.HealingSimulation))
        g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Healing Simulation of " + r["simulacrum"].strip() + " in a " + r["cc"] + " context as a cure for " + rco.strip())))
        g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
        g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
        g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
        g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.healedRealityCounterpart, URIRef(dizuri+rccorr)))
        g.add((URIRef(dizuri+rccorr), ont.isHealedRealityCounterpartOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
        g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        if len(rco) > thold:
          longrealitycounterparts.append(realitycounterp)
  elif "antidote for " in r["rc"]:
    cureforlist2 = r["rc"].split("antidote for ", 1)[1].split(",")
    for cure in cureforlist2:
      rco = cure
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
        g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
        simulation = r["simulacrumc"]+"-"+rccorr
        g.add((URIRef(dizuri+simulation), RDF.type, ont.HealingSimulation))
        g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Healing Simulation of " + r["simulacrum"].strip() + " in a " + r["cc"] + " context as a cure for " + rco.strip())))
        g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
        g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
        g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
        g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.healedRealityCounterpart, URIRef(dizuri+rccorr)))
        g.add((URIRef(dizuri+rccorr), ont.isHealedRealityCounterpartOf, URIRef(dizuri+simulation)))
        g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
        g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        if len(rco) > thold:
          longrealitycounterparts.append(realitycounterp)
  elif "charm" in r["rc"]:
    found = False
    for typeofcharm in charmmapping:
      if typeofcharm[0] in r["rc"]:
        charmlist = r["rc"].split(typeofcharm[0], 1)[1].split(",")
        found = True
        for charm in charmlist:
          rco = charm
          rccorr = camel(rco.strip().replace("’", "'"))
          if "see @" not in rco:
            g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
            g.add((URIRef(dizuri+"charm"), RDF.type, ont.RealityCounterpart))
            g.add((URIRef(dizuri+"charm"), RDFS.label, Literal("charm")))
            g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
            simulation = r["simulacrumc"]+"-"+"Charm"+"-"+rccorr
            g.add((URIRef(dizuri+simulation), RDF.type, URIRef(onturi+typeofcharm[1]["simtype"])))
            g.add((URIRef(dizuri+simulation), RDFS.label, Literal(typeofcharm[1]["litsimtype"] + " of a "+ typeofcharm[0] + " " + rco.strip() + " in a " + r["cc"] + " context as "  + r["simulacrum"].strip() )))
            g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
            g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
            g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
            g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
            g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
            g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+"charm")))
            g.add((URIRef(dizuri+"charm"), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
            g.add((URIRef(dizuri+simulation), URIRef(onturi+typeofcharm[1]["additionalrel"]), URIRef(dizuri+rccorr)))
            g.add((URIRef(dizuri+rccorr), URIRef(onturi+typeofcharm[1]["ari"]), URIRef(dizuri+simulation)))
            g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
            g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
    if found is False:
      rco = r["rc"]    
      rccorr = camel(rco.strip().replace("’", "'"))
      if "see @" not in rco:
        if ":" not in rco:
          g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
          g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
          simulation = r["simulacrumc"]+"-"+rccorr
          g.add((URIRef(dizuri+simulation), RDF.type, ont.Simulation))
          g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Simulation of " + rco.strip() + " in a " + r["cc"] + " context as " + r["simulacrum"].strip())))
          g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
          g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
          g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
          g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
          g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
          g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+rccorr)))
          g.add((URIRef(dizuri+rccorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
          g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
          g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
        else:
          newcontextorvariant = rco.split(":")[0]
          content = rco.split(":")[1]
          if "," in content:
            content = content.split(",")
          else:
            content = [content]
          if newcontextorvariant in realcontexts:
            contextcorr = camel(newcontextorvariant)
            for specialrc in content:
              specialrcc = camel(specialrc)
              g.add((URIRef(dizuri+specialrcc), RDF.type, ont.RealityCounterpart))
              g.add((URIRef(dizuri+specialrcc), RDFS.label, Literal(specialrc.strip())))
              simulation = r["simulacrumc"]+"-"+specialrcc
              g.add((URIRef(dizuri+simulation), RDF.type, ont.Simulation))
              g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Simulation of " + specialrc.strip() + " in a " + newcontextorvariant.strip() + " context as " + r["simulacrum"].strip())))
              g.add((URIRef(dizuri+contextcorr), RDF.type, ont.Context))
              g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
              g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
              g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
              g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+specialrcc)))
              g.add((URIRef(dizuri+specialrcc), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
              g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
              g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
          else:
            variantcorr = camel(newcontextorvariant)
            g.add((URIRef(dizuri+variantcorr), RDF.type, ont.Simulacrum))
            g.add((URIRef(dizuri+variantcorr), RDFS.label, Literal(newcontextorvariant.strip())))
            g.add((URIRef(dizuri+variantcorr), ont.isVariantOf, URIRef(dizuri+r["simulacrumc"])))
            g.add((URIRef(dizuri+r["simulacrumc"]), ont.hasVariant, URIRef(dizuri+variantcorr)))
            for specialrc in content:
              specialrcc = camel(specialrc)
              g.add((URIRef(dizuri+specialrcc), RDF.type, ont.RealityCounterpart))
              g.add((URIRef(dizuri+specialrcc), RDFS.label, Literal(specialrc.strip())))
              simulation = variantcorr+"-"+specialrcc
              g.add((URIRef(dizuri+simulation), RDF.type, ont.Simulation))
              g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Simulation of " + specialrc.strip() + " in a " + r["cc"] + " context as " + newcontextorvariant.strip())))
              g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
              g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
              g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
              g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+variantcorr)))
              g.add((URIRef(dizuri+variantcorr), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
              g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+specialrcc)))
              g.add((URIRef(dizuri+specialrcc), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
              g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
              g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))   
    

    if len(rco) > thold:
      longrealitycounterparts.append(realitycounterp)
  else:
#specificrel.append({"rc":realitycounterp.strip(), "context":contextcorr, "simulacrum":generalvariant.strip(), "simulacrumc":generalvariantcorr})
        rco = r["rc"]
        rccorr = camel(rco.strip().replace("’", "'"))
        if "see @" not in rco:
          if ":" not in rco:
            g.add((URIRef(dizuri+rccorr), RDF.type, ont.RealityCounterpart))
            g.add((URIRef(dizuri+rccorr), RDFS.label, Literal(rco.strip())))
            simulation = r["simulacrumc"]+"-"+rccorr
            g.add((URIRef(dizuri+simulation), RDF.type, ont.Simulation))
            g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Simulation of " + rco.strip() + " in a " + r["cc"] + " context as " + r["simulacrum"].strip())))
            g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
            g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
            g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
            g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
            g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
            g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+rccorr)))
            g.add((URIRef(dizuri+rccorr), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
            g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
            g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
          else:
            newcontextorvariant = rco.split(":")[0]
            content = rco.split(":")[1]
            if "," in content:
              content = content.split(",")
            else:
              content = [content]
            if newcontextorvariant in realcontexts:
              contextcorr = camel(newcontextorvariant)
              for specialrc in content:
                specialrcc = camel(specialrc)
                g.add((URIRef(dizuri+specialrcc), RDF.type, ont.RealityCounterpart))
                g.add((URIRef(dizuri+specialrcc), RDFS.label, Literal(specialrc.strip())))
                simulation = r["simulacrumc"]+"-"+specialrcc
                g.add((URIRef(dizuri+simulation), RDF.type, ont.Simulation))
                g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Simulation of " + specialrc.strip() + " in a " + newcontextorvariant.strip() + " context as " + r["simulacrum"].strip())))
                g.add((URIRef(dizuri+contextcorr), RDF.type, ont.Context))
                g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+r["simulacrumc"])))
                g.add((URIRef(dizuri+r["simulacrumc"]), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+specialrcc)))
                g.add((URIRef(dizuri+specialrcc), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
                g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))
            else:
              variantcorr = camel(newcontextorvariant)
              g.add((URIRef(dizuri+variantcorr), RDF.type, ont.Simulacrum))
              g.add((URIRef(dizuri+variantcorr), RDFS.label, Literal(newcontextorvariant.strip())))
              g.add((URIRef(dizuri+variantcorr), ont.isVariantOf, URIRef(dizuri+r["simulacrumc"])))
              g.add((URIRef(dizuri+r["simulacrumc"]), ont.hasVariant, URIRef(dizuri+variantcorr)))
              for specialrc in content:
                specialrcc = camel(specialrc)
                g.add((URIRef(dizuri+specialrcc), RDF.type, ont.RealityCounterpart))
                g.add((URIRef(dizuri+specialrcc), RDFS.label, Literal(specialrc.strip())))
                simulation = variantcorr+"-"+specialrcc
                g.add((URIRef(dizuri+simulation), RDF.type, ont.Simulation))
                g.add((URIRef(dizuri+simulation), RDFS.label, Literal("Simulation of " + specialrc.strip() + " in a " + r["cc"] + " context as " + newcontextorvariant.strip())))
                g.add((URIRef(dizuri+simulation), ont.hasContext, URIRef(dizuri+r["context"])))
                g.add((URIRef(dizuri+r["context"]), RDF.type, ont.Context))
                g.add((URIRef(dizuri+r["context"]), ont.isContextOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasSimulacrum, URIRef(dizuri+variantcorr)))
                g.add((URIRef(dizuri+variantcorr), ont.isSimulacrumOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasRealityCounterpart, URIRef(dizuri+specialrcc)))
                g.add((URIRef(dizuri+specialrcc), ont.isRealityCounterpartOf, URIRef(dizuri+simulation)))
                g.add((URIRef(dizuri+simulation), ont.hasSource, diz.DictionaryOfSymbols))
                g.add((diz.DictionaryOfSymbols, ont.isSourceOf, URIRef(dizuri+simulation)))

            



In [ ]:
g.serialize(destination = "yourfilename.ttl", format="turtle")